In [ ]:
import os
os.environ[SQLALCHEMY_DATABASE_URI] = “sqlite:///file:test_db?mode=memory&cache=shared&uri=true”

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
import numpy as np
import pandas as pd
import torch
import gc
import syft as sy
from syft.core.adp.entity import Entity
from pympler.asizeof import asizeof #pip install pympler
from syft import serialize
from syft import deserialize

import timeit
from syft.core.tensor.autodp.dp_tensor_converter import convert_to_gamma_tensor

from functools import reduce

In [10]:
df = pd.read_parquet("1M_rows_dataset_sample.parquet")
print("Number of Rows: ",df.shape[0])
df.head()
df = df[:5]
print(df.shape[0])

name = "Tweets- 100000 rows dataset "
impressions = ((np.array(list(df['impressions'])))).astype(np.int32)
publication_title = ((list(df['user_id'])))

entities = list()
for i in range(len(publication_title)):
    entities.append(Entity(name=str(publication_title[i])))
    
tweets_data = sy.Tensor(impressions).private(min_val=0, max_val=30, entities = entities)

Number of Rows:  1000000
5


In [11]:
# %%time
left_result = tweets_data.sum()

In [12]:
from syft.core.adp.adversarial_accountant import AdversarialAccountant
from nacl.signing import VerifyKey, SigningKey

In [13]:
key = SigningKey.generate()
the_actual_key = key.verify_key

In [14]:
acc = AdversarialAccountant()
left_result.publish(sigma=1000, acc=acc, user_key=the_actual_key)

AttributeError: 'dict' object has no attribute 'query'